# Linear, ridge, and lasso regression example

## Import the libraries

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

## Load the data

In [ ]:
data = pd.read_csv('Hitters.csv')
df_hitters = data.copy()
df_hitters

In [ ]:
# Categorical variables
print('The league types are:', df_hitters['League'].unique())
print('The divison types are:', df_hitters['Division'].unique())
print('The new league options are:', df_hitters['NewLeague'].unique())

In [ ]:
df_hitters_num = pd.get_dummies(df_hitters, columns = ['League', 'Division', 'NewLeague'], drop_first=True)
df_hitters_num

In [ ]:
# Check if there are NaN values
df_hitters_num.isnull().sum()

In [ ]:
df_hitters_num_nonull = df_hitters_num.dropna()
df_hitters_num_nonull.isnull().sum()

In [ ]:
# Check distribution
sns.displot(df_hitters_num_nonull['Salary']);

In [ ]:
# Check the correlation between the dependent and independent variables
correlation = df_hitters_num_nonull.corr()
correlation['Salary'].sort_values(ascending=True)

In [ ]:
# Check for multicolinearity
plt.figure(figsize=(11,9))
sns.heatmap(df_hitters.corr(),
            vmin = -1, 
            vmax = 1,
            cmap ="GnBu",
            annot=True)
plt.show()

## Declare the dependent and independent variables

In [ ]:
X = df_hitters_num_nonull.drop('Salary', axis = 1)
y = df_hitters_num_nonull['Salary']

## Split the data into training and testing parts

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=365)

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Perform linear regression

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

In [ ]:
print("Linear Regression coefficients are: ",lin_reg.coef_)
print("Linear Regression y-intercept is: ",lin_reg.intercept_)

In [ ]:
lin_reg_y_pred = lin_reg.predict(X_test)
lin_reg_y_pred

In [ ]:
lin_comp = pd.DataFrame({'Predicted': lin_reg_y_pred, 'Actual': y_test})
lin_comp

In [ ]:
print("Linear Regression Model RMSE is: ", math.sqrt(mean_squared_error(y_test, lin_reg_y_pred)))
print("Linear Regression Model Training Score: ",lin_reg.score(X_train, y_train))
print("Linear Regression Model Testing Score: ",lin_reg.score(X_test, y_test))

## Perform ridge regression 

In [ ]:
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

In [ ]:
# Initializing the ridge regressor
ridge = RidgeCV(alphas=np.arange(0.1, 10, 0.1), cv=cv, scoring='neg_mean_absolute_error')

In [ ]:
# Fitting the ridge regressor
ridge.fit(X_train,y_train)
ridge_reg_y_pred = ridge.predict(X_test)

print("Ridge tuning parameter:", (ridge.alpha_))
print ("Ridge model coefficients:", (ridge.coef_))
print ("Ridge model intercept:", (ridge.intercept_))

In [ ]:
print("Ridge Regression Model RMSE is: ", math.sqrt(mean_squared_error(y_test, ridge_reg_y_pred)))
print("Ridge Regression Model Training Score: ",ridge.score(X_train, y_train))
print("Ridge Regression Model Testing Score: ",ridge.score(X_test, y_test))

## Perform lasso regression

In [ ]:
# Initializing the lasso regressor
lasso = LassoCV(alphas=np.arange(0.1, 10.0, 0.1), cv=cv, tol = 1)

In [ ]:
# Fitting the lasso regressor
lasso.fit(X_train,y_train)
lasso_reg_y_pred = lasso.predict(X_test)

print("Lasso tuning parameter:", (lasso.alpha_))
print ("Lasso model coefficients:", (lasso.coef_))
print ("Lassso model intercept:", (lasso.intercept_))

In [ ]:
print("Lasso Regression Model RMSE is: ", math.sqrt(mean_squared_error(y_test, lasso_reg_y_pred)))
print("Lasso Regression Model Training Score: ",lasso.score(X_train, y_train))
print("Lasso Regression Model Testing Score: ",lasso.score(X_test, y_test))

## Compare the score

In [ ]:
print("Linear Regression Model Training Score: ",lin_reg.score(X_train, y_train))
print("Linear Regression Model Testing Score: ",lin_reg.score(X_test, y_test))
print("Ridge Regression Model Training Score: ",ridge.score(X_train, y_train))
print("Ridge Regression Model Testing Score: ",ridge.score(X_test, y_test))
print("Lasso Regression Model Training Score: ",lasso.score(X_train, y_train))
print("Lasso Regression Model Testing Score: ",lasso.score(X_test, y_test))

## Root mean squared error

In [ ]:
print("Linear Regression Model RMSE is: ", math.sqrt(mean_squared_error(y_test, lin_reg_y_pred)))
print("Ridge Regression Model RMSE is: ", math.sqrt(mean_squared_error(y_test, ridge_reg_y_pred)))
print("Lasso Regression Model RMSE is: ", math.sqrt(mean_squared_error(y_test, lasso_reg_y_pred)))

## Replacing the missing values in the DataFrame

In [ ]:
df_hitters_nan = df_hitters_num[df_hitters_num['Salary'].isnull()]
df_hitters_nan

In [ ]:
X_nan = df_hitters_nan.drop('Salary', axis = 1)
y_nan = df_hitters_nan['Salary']

In [ ]:
scaler = StandardScaler()
X_nan = scaler.fit_transform(X_nan)

In [ ]:
nan_pred = ridge.predict(X_nan)
nan_pred

In [ ]:
# df_hitters_nan.drop(['Salary'], axis=1)
df_nan_full = df_hitters_nan.copy()
df_nan_full['Salary'] = nan_pred
df_nan_full